In [2]:
import numpy as np
import h5py
import os
from scipy.spatial import cKDTree

def deg_to_cartesian(ra, dec):
    ra = np.radians(ra)
    dec = np.radians(dec)
    return np.cos(ra) * np.cos(dec), np.sin(ra) * np.cos(dec), np.sin(dec)

tolerance = 1/3600
tolerance_rad = np.radians(tolerance)

h5_dir = '/arc/projects/ots/HSC_h5/'

#filenames = ['HSC_dud_galaxy_calexp_GIRYZ7610_64.h5', 
#             'HSC_dud_qso_calexp_GIRYZ7610_64.h5',
#             'HSC_dud_star_calexp_GIRYZ7610_64.h5',
#             'HSC_dud_dwarf_galaxy_calexp_GIRYZ7610_64.h5',
#             'HSC_dud_unknown_calexp_GIRYZ7610_64.h5']
filenames = ['HSC_dud_galaxy_GIRYZ7610_64.h5', 
             'HSC_dud_qso_GIRYZ7610_64.h5',
             'HSC_dud_star_GIRYZ7610_64.h5',
             'HSC_dud_dwarf_galaxy_GIRYZ7610_64.h5',
             'HSC_dud_unknown_GIRYZ7610_64.h5']

# Store Cartesian coordinates and original indices
xyz = []
original_indices = []
for fn in filenames:
    with h5py.File(os.path.join(h5_dir, fn), "r") as f:
        ra = f['ra'][:]
        dec = f['dec'][:]
        x, y, z = deg_to_cartesian(ra, dec)
        xyz.append(np.vstack((x, y, z)).T)
        original_indices.append(np.arange(len(ra)))  # Original indices

# Function to remove duplicates within each dataset
def remove_duplicates_within(data, tolerance_rad):
    tree = cKDTree(data)
    indices_to_keep = []
    indices_already_checked = set()
    
    for i in range(data.shape[0]):
        if i in indices_already_checked:
            continue
        # Find points within tolerance
        indices_within = tree.query_ball_point(data[i], r=tolerance_rad)
        indices_to_keep.append(indices_within[0])  # Keep the first index
        indices_already_checked.update(indices_within)
        
    return np.array(indices_to_keep)

# Step 1: Remove duplicates within each dataset and collect good indices
good_indices_within = [remove_duplicates_within(data, tolerance_rad) for data in xyz]

# Step 2: Remove matches across datasets
final_good_indices = [good_indices.copy() for good_indices in good_indices_within]

for i in range(len(xyz)):
    for j in range(i + 1, len(xyz)):
        if len(final_good_indices[i]) == 0 or len(final_good_indices[j]) == 0:
            continue
        
        data_i = xyz[i][final_good_indices[i]]
        data_j = xyz[j][final_good_indices[j]]
        
        tree_i = cKDTree(data_i)
        tree_j = cKDTree(data_j)
        
        matches_i = tree_i.query_ball_tree(tree_j, r=tolerance_rad)
        matches_j = tree_j.query_ball_tree(tree_i, r=tolerance_rad)
        
        # Indices to keep (those not in matches)
        keep_i = set(range(len(data_i))) - set(np.concatenate(matches_i))
        keep_j = set(range(len(data_j))) - set(np.concatenate(matches_j))
        
        # Update final good indices after cross-dataset check
        final_good_indices[i] = final_good_indices[i][list(keep_i)]
        final_good_indices[j] = final_good_indices[j][list(keep_j)]

# final_good_indices contains lists of indices for each dataset that are considered "good"
# These indices can be used to extract the "good" points from the original datasets

In [3]:
for i, fn in enumerate(filenames):
    original_path = os.path.join(h5_dir, fn)
    new_path = os.path.join(h5_dir, fn.replace('.h5', '_new.h5'))
    
    with h5py.File(original_path, 'r') as original_file, h5py.File(new_path, 'w') as new_file:
        good_indices = final_good_indices[i]
        
        for dataset_name in original_file.keys():
            original_dataset = original_file[dataset_name]
            shape = original_dataset.shape
            dtype = original_dataset.dtype
            
            # Adjust the shape for the new dataset based on the number of good indices
            new_shape = (len(good_indices),) + shape[1:]
            new_dataset = new_file.create_dataset(dataset_name, shape=new_shape, dtype=dtype)
            
            # Copy attributes from the original dataset to the new one
            for attr_name, attr_value in original_dataset.attrs.items():
                new_dataset.attrs[attr_name] = attr_value
            
            # Write each good data point to the new dataset one by one
            for j, index in enumerate(good_indices):
                new_dataset[j] = original_dataset[index]

In [4]:
1/0.8

1.25

In [12]:
print([len(_) for _ in xyz])
print([len(_) for _ in good_indices_within])
print([len(_) for _ in final_good_indices])

[147927, 23366, 59896, 8, 437154]
[53853, 9281, 25598, 8, 199766]
[53434, 9250, 25586, 8, 197906]


In [1]:
import numpy as np
import h5py
import os
from scipy.spatial import cKDTree

def deg_to_cartesian(ra, dec):
    # Convert RA and DEC to radians for spatial indexing
    ra = np.radians(ra)
    dec = np.radians(dec)
    # Convert to Cartesian coordinates
    return np.cos(ra) * np.cos(dec), np.sin(ra) * np.cos(dec), np.sin(dec)

tolerance = 1/3600  # Tolerance in degrees
tolerance_rad = np.radians(tolerance)  # Convert tolerance to radians

h5_dir = '/arc/projects/ots/HSC_h5/'

filenames = ['HSC_dud_galaxy_calexp_GIRYZ7610_64.h5', 
             'HSC_dud_qso_calexp_GIRYZ7610_64.h5',
             'HSC_dud_star_calexp_GIRYZ7610_64.h5',
             'HSC_dud_dwarf_galaxy_calexp_GIRYZ7610_64.h5',
             'HSC_dud_unknown_calexp_GIRYZ7610_64.h5']

xyz = []
for fn in filenames:
    with h5py.File(os.path.join(h5_dir, fn), "r") as f:
        ra = f['ra'][:]
        dec = f['dec'][:]
    
        # Convert RA and Dec to Cartesian
        x, y, z = deg_to_cartesian(ra, dec)
        xyz.append(np.vstack((x, y, z)).T)

In [4]:
h5_dir = '/arc/projects/ots/HSC_h5/'

filenames = ['HSC_dud_galaxy_calexp_GIRYZ7610_64.h5', 
             'HSC_dud_qso_calexp_GIRYZ7610_64.h5',
             'HSC_dud_star_calexp_GIRYZ7610_64.h5',
             'HSC_dud_dwarf_galaxy_calexp_GIRYZ7610_64.h5',
             'HSC_dud_unknown_calexp_GIRYZ7610_64.h5']

xyz = []
for fn in filenames:
    with h5py.File(os.path.join(h5_dir, fn), "r") as f:
        ra = f['ra'][:]
        dec = f['dec'][:]
    
        # Convert RA and Dec to Cartesian
        x, y, z = deg_to_cartesian(ra, dec)
        xyz.append(np.vstack((x, y, z)).T)

In [5]:
xyz[0].shape

(147927, 3)

In [21]:
64/(1000/256 )

16.384